# XML exercise

Using data from [**mondial database**](https://drive.google.com/file/d/14lFT4nWHgwN36ij4XZh6OUuup-K9qLgR/view?usp=sharing) find the answers to following questions:

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [3]:
import pandas as pd
import xml.etree.ElementTree as ET

In [4]:
tree = ET.parse('mondial.xml')
tree

In [5]:
root = tree.getroot()
root

<Element 'mondial' at 0x00000206A0CBA810>

In [6]:
print(root.tag)
print(root.attrib)
print(len(root))

mondial
{}
3403


In [7]:
monde = pd.read_xml('mondial.xml')
monde.head()

,car_code,area,capital,memberships,name,population,population_growth,infant_mortality,gdp_total,gdp_agri,...,height,sea,islands,river,lake,last_eruption,mountains,iatacode,gmtOffset,located_on
0,AL,28750.0,cty-Albania-Tirane,org-BSEC org-CEI org-CD org-SELEC org-CE org-E...,Albania,2821977.0,0.30,13.19,12800.0,19.5,...,NaN,None,None,None,None,None,None,None,NaN,NaN
1,GR,131940.0,cty-Greece-Athens,org-AG org-BIS org-BSEC org-CD org-SELEC org-C...,Greece,10816286.0,0.01,4.78,243300.0,3.5,...,NaN,None,None,None,None,None,None,None,NaN,NaN
2,MK,25333.0,cty-Macedonia-Skopje,org-BIS org-CEI org-CD org-SELEC org-CE org-EA...,North Macedonia,2059794.0,0.21,7.90,10650.0,10.2,...,NaN,None,None,None,None,None,None,None,NaN,NaN
3,SRB,77474.0,city-Belgrade-SRB-SRB,org-BIS org-BSEC org-CEI org-CD org-SELEC org-...,Serbia,7120666.0,-0.46,6.16,43680.0,7.9,...,NaN,None,None,None,None,None,None,None,NaN,NaN
4,MNE,14026.0,city-Podgorica-MNE-MNE,org-CEI org-CD org-SELEC org-CE org-EAPC org-E...,Montenegro,620029.0,-0.49,NaN,4518.0,0.8,...,NaN,None,None,None,None,None,None,None,NaN,NaN


## 10 countries with the lowest infant mortality rates

In [19]:
inf_mort = monde.sort_values('infant_mortality').head(10)
inf_mort[['name','infant_mortality']]

,name,infant_mortality
38,Monaco,1.81
98,Japan,2.13
117,Bermuda,2.48
36,Norway,2.48
106,Singapore,2.53
37,Sweden,2.60
10,Czech Republic,2.63
8,Spain,2.70
78,Hong Kong,2.73
79,Macao,3.13


In [62]:
#MENTOR ANSWER
#not sure why the numbers are so different

import pandas as pd  
inf_data = {} 

for element in tree.iterfind('country'):     
    this_element= {}     
    this_element["infant_mortality"] = element.findtext('infant_mortality')     
    inf_data[element.findtext('name')] = this_element
    
inf_data_pd = pd.DataFrame(inf_data).transpose() 
inf_data_pd.sort_values("infant_mortality")[:10]

,infant_mortality
Monaco,1.81
Romania,10.16
Fiji,10.2
Brunei,10.48
Grenada,10.5
Mauritius,10.59
Panama,10.7
Seychelles,10.77
United Arab Emirates,10.92
Barbados,10.93


## 10 cities with the largest population

In [20]:
monde.columns

Index(['car_code', 'area', 'capital', 'memberships', 'name', 'population',
       'population_growth', 'infant_mortality', 'gdp_total', 'gdp_agri',
       'gdp_ind', 'gdp_serv', 'inflation', 'unemployment', 'indep_date',
       'government', 'encompassed', 'ethnicgroup', 'religion', 'language',
       'border', 'city', 'localname', 'province', 'dependent', 'id', 'headq',
       'abbrev', 'established', 'members', 'country', 'bordering', 'located',
       'depth', 'to', 'length', 'source', 'estuary', 'island', 'through',
       'latitude', 'longitude', 'elevation', 'type', 'salinity', 'height',
       'sea', 'islands', 'river', 'lake', 'last_eruption', 'mountains',
       'iatacode', 'gmtOffset', 'located_on'],
      dtype='object')

In [58]:
monde[['capital','population']].head(2)
#but these populations are for the country not the cities, we need to find the city pop within the .xml

,capital,population
0,cty-Albania-Tirane,2821977.0
1,cty-Greece-Athens,10816286.0


In [63]:
#MENTOR SOLUTION
city_data = {} 

for element in tree.iterfind('country'):          #iternation find, looks for country
    cities = element.findall('city')              #now find all the cities within that country
    if len(cities) == 0:                          #if you have cities within this country
        cities = element.findall("province/city") #inside the 2nd level city, look for prov then city in one step     there is a city in 2nd and 3rd level
        for subelement in cities:                 #subelement is prov or city name.
            this_element= {}                      #empty dict where we will store the following data for each prov/city
            try:                                  #use try bc there may be null values. 
                this_element["population"] = int(subelement.findall('population')[-1].text)       #store all populations in dict "this_element", then grab the last one [-1]. Subelement = the city you are looking at. .text is grabbing the actual value
            except:                               # skip if null 
                pass             
                #print element.findall('city')                           
            city_data[subelement.findtext('name')] = this_element              #city_data is the home for your city : pop. "subelement.findtext('name')" defining your keys, "this_element" = value (population)


In [69]:
city_data_pd = pd.DataFrame(city_data).transpose() 
city_data_pd.head()

,population
Komotini,NaN
Kavala,58790.0
Athina,664046.0
Peiraias,163688.0
Peristeri,139981.0


In [66]:
#THE SOLUTION
city_data_pd.sort_values("population", ascending=False)[:10]

,population
Shanghai,22315474.0
Karachi,14916456.0
Lagos,13745000.0
Istanbul,13710512.0
Mumbai,12442373.0
Moskva,11979529.0
Beijing,11716620.0
Kinshasa,11575000.0
São Paulo,11152344.0
Lahore,11126285.0


## name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [12]:
#print list of col headers
monde.columns

Index(['car_code', 'area', 'capital', 'memberships', 'name', 'population',
       'population_growth', 'infant_mortality', 'gdp_total', 'gdp_agri',
       'gdp_ind', 'gdp_serv', 'inflation', 'unemployment', 'indep_date',
       'government', 'encompassed', 'ethnicgroup', 'religion', 'language',
       'border', 'city', 'localname', 'province', 'dependent', 'id', 'headq',
       'abbrev', 'established', 'members', 'country', 'bordering', 'located',
       'depth', 'to', 'length', 'source', 'estuary', 'island', 'through',
       'latitude', 'longitude', 'elevation', 'type', 'salinity', 'height',
       'sea', 'islands', 'river', 'lake', 'last_eruption', 'mountains',
       'iatacode', 'gmtOffset', 'located_on'],
      dtype='object')

### A) longest river

In [79]:
#longest river

rivers = {}

for element in tree.iterfind('river'):                                                    # <ELEMENT> elements are in <> in xml. Look for any element with river
    try:
        rivers[element.findtext('name')] = {'length': float(element.findtext('length')), 
                                            'country': element.attrib['country']}
    except:
        pass
    
rivers_pd = pd.DataFrame(rivers).transpose() 
rivers_pd.head()

,length,country
Thjorsa,230.0,IS
Jökulsa a Fjöllum,206.0,IS
Thames,346.0,GB
Severn,354.0,GB
Trent,298.0,GB


In [81]:
#THE SOLUTION
rivers_pd.sort_values("length", ascending=False)[:1]

,length,country
Yangtze,6380.0,CN


### Largest lake

In [82]:
lakes = {}

for element in tree.iterfind('lake'):                                                    # <ELEMENT> elements are in <> in xml. Look for any element with lake
    try:
        lakes[element.findtext('name')] = {'area': float(element.findtext('area')), 
                                            'country': element.attrib['country']}
    except:
        pass
    
lakes_pd = pd.DataFrame(lakes).transpose() 
lakes_pd.head()

,area,country
Inarijärvi,1040.0,SF
Oulujärvi,928.0,SF
Saimaa,4370.0,SF
Päijänne,1118.0,SF
Mjoesa-See,368.0,N


In [83]:
#THE SOLUTION
lakes_pd.sort_values("area", ascending=False)[:1]

,area,country
Caspian Sea,386400.0,R AZ KAZ IR TM


### airport at highest elevation

In [84]:
airp = {}

for element in tree.iterfind('airport'):                                                    # <ELEMENT> elements are in <> in xml. Look for any element with airport
    try:
        airp[element.findtext('name')] = {'elevation': float(element.findtext('elevation')), 
                                            'country': element.attrib['country']}
    except:
        pass
    
airp_pd = pd.DataFrame(airp).transpose() 
airp_pd.head()

,elevation,country
Herat,977.0,AFG
Kabul Intl,1792.0,AFG
Tirana Rinas,38.0,AL
Cheikh Larbi Tebessi,811.0,DZ
Batna Airport,822.0,DZ


In [85]:
#THE SOLUTION
airp_pd.sort_values("elevation", ascending=False)[:1]

,elevation,country
El Alto Intl,4063.0,BOL
